In [1]:
import json
from datetime import datetime
import pandas as pd

## Read File

In [2]:
with open('trainings (correct).txt') as file:
    data = json.load(file)

### List each completed training with a count of how many people have completed that training.

In [3]:
# Conver the json file to data frame in Pandas format
df = pd.DataFrame(data)
df

,name,completions
0,Jaelyn Quinn,"[{'name': 'Electrical Safety for Labs', 'times..."
1,Asia Duke,"[{'name': 'X-Ray Safety', 'timestamp': '9/1/20..."
2,Cason Gross,[{'name': 'Using Hazardous Chemicals in an Ani...
3,Hector Dixon,[{'name': 'Awareness Training for the Transpor...
4,Josephine Cantrell,[{'name': 'Transportation of Infectious Substa...
...,...,...
995,Shaylee Ware,[{'name': 'Physical Science Responsible Conduc...
996,Jayda Roman,[{'name': 'Occupational Exposure to Bloodborne...
997,America Franco,"[{'name': 'Understanding Biosafety', 'timestam..."
998,Paxton Foley,"[{'name': 'Understanding Biosafety', 'timestam..."


In [4]:
# Normalize the dataframe for 'completions' column which is a nested list of dictionary
df = pd.json_normalize(data, record_path='completions', record_prefix='training_', meta=['name'])

In [5]:
df

,training_name,training_timestamp,training_expires,name
0,Electrical Safety for Labs,8/31/2022,None,Jaelyn Quinn
1,Safe Handling of Human Cell Lines/Materials in...,10/30/2023,None,Jaelyn Quinn
2,Awareness Training for the Transport of Hazard...,10/5/2023,None,Jaelyn Quinn
3,X-Ray Safety,4/8/2023,None,Jaelyn Quinn
4,X-Ray Safety,9/1/2022,None,Asia Duke
...,...,...,...,...
2962,Physical Science Responsible Conduct of Resear...,6/10/2023,None,Jadon Harrell
2963,Animal Care And Use Risk Assessment,8/22/2023,8/21/2024,Jadon Harrell
2964,OHS Training,3/14/2022,3/14/2023,Jadon Harrell
2965,Occupational Exposure to Bloodborne Pathogens,11/3/2022,11/3/2023,Jadon Harrell


In [6]:
# Print the number of people who have completed the training (only considering as one count if a person completes the training more than once).
result_1 = df.groupby('training_name')['name'].nunique().to_dict()

In [7]:
with open('output_1_training_count.json', 'w') as outfile:
    json.dump(result_1, outfile, indent=4)

### Given a list of trainings and a fiscal year (defined as 7/1/n-1 – 6/30/n), for each specified training, list all people that completed that training in the specified fiscal year.
* Use parameters: Trainings = "Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training"; Fiscal Year = 2024

In [8]:
# Convert the 'training_timestamp to datetime format
df['training_timestamp'] = pd.to_datetime(df['training_timestamp'], format="%m/%d/%Y")

In [9]:
# Given the parameter year
parameter_year = 2024

In [10]:
# Specify the start and end date
start_date = datetime(parameter_year - 1, 7, 1)
end_date = datetime(parameter_year, 6, 30)
# Filter the dataframe based on the start and end date 
filtered_df = df[(df['training_timestamp'] >= start_date) &
(df['training_timestamp'] <= end_date)]


In [11]:
trainings_of_interest = ["Electrical Safety for Labs", "X-Ray Safety", "Laboratory Safety Training"]
df_filtered = filtered_df[filtered_df['training_name'].isin(trainings_of_interest)]
result_2 = df_filtered.groupby('training_name')['name'].unique()
# Convert the lists of names to lists of dictionaries with a "name" key
result_2_serializable = {k: {"name": ", ".join(v)} for k, v in result_2.items()}

with open('output_2_training_names.json', 'w') as outfile:
    json.dump(result_2_serializable, outfile, indent=4)

### Given a date, find all people that have any completed trainings that have already expired, or will expire within one month of the specified date (A training is considered expired the day after its expiration date). For each person found, list each completed training that met the previous criteria, with an additional field to indicate expired vs expires soon.
* Use date: Oct 1st, 2023

In [12]:
expiration_date = datetime(2023, 10, 1)
within_one_month = datetime(2023, 9, 1)
# Convert 'traning_expires' to datetime format
df['training_expires'] = pd.to_datetime(df['training_expires'], format="%m/%d/%Y")
df

,training_name,training_timestamp,training_expires,name
0,Electrical Safety for Labs,2022-08-31,NaT,Jaelyn Quinn
1,Safe Handling of Human Cell Lines/Materials in...,2023-10-30,NaT,Jaelyn Quinn
2,Awareness Training for the Transport of Hazard...,2023-10-05,NaT,Jaelyn Quinn
3,X-Ray Safety,2023-04-08,NaT,Jaelyn Quinn
4,X-Ray Safety,2022-09-01,NaT,Asia Duke
...,...,...,...,...
2962,Physical Science Responsible Conduct of Resear...,2023-06-10,NaT,Jadon Harrell
2963,Animal Care And Use Risk Assessment,2023-08-22,2024-08-21,Jadon Harrell
2964,OHS Training,2022-03-14,2023-03-14,Jadon Harrell
2965,Occupational Exposure to Bloodborne Pathogens,2022-11-03,2023-11-03,Jadon Harrell


In [13]:
df_expire = df[(df['training_expires'] < expiration_date) | 
        ((df['training_expires'] >= within_one_month) & (df['training_expires'] < expiration_date))]
df_expire

,training_name,training_timestamp,training_expires,name
38,Using Hazardous Chemicals in an Animal Care Fa...,2022-09-01,2023-09-01,Muhammad Kaufman
43,IRB Quiz,2022-01-20,2023-01-20,Lexie Mckinney
45,Safe Handling of Human Cell Lines/Materials in...,2022-08-10,2023-08-10,Gretchen Boyer
49,Awareness Training for the Transport of Hazard...,2022-06-27,2023-06-27,Denise Allen
52,Understanding Biosafety,2022-01-29,2023-01-29,Denise Allen
...,...,...,...,...
2900,"Transportation of Infectious Substances, Categ...",2022-02-23,2023-02-23,Julien Terrell
2911,Basic Training Program for Animal Users,2022-09-04,2023-09-04,John Montes
2913,Radioactive Materials Safety Training,2022-03-15,2023-03-15,John Montes
2940,Working in Cold Temperatures,2022-08-27,2023-08-27,Camden Ho


In [14]:
# Add a new column to indicate expired vs expires soon
df_expire['status'] = df_expire['training_expires'].apply(
    lambda x: 'expired' if x < expiration_date else 'expires soon'
)

# Group by person and list each completed training with the status
result_3 = df_expire.groupby('name').apply(
    lambda x: x[['training_name', 'status']].to_dict(orient='records')
).to_dict()

with open('output_3_expired_or_expires_soon.json', 'w') as outfile:
    json.dump(result_3, outfile, indent=4)

/var/folders/hd/1_05khmx18g4v9p82m039_9c0000gn/T/ipykernel_64964/2322672614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_expire['status'] = df_expire['training_expires'].apply(
